In [1]:
import re
from collections import OrderedDict
import numpy as np

class sentistrength:
    def __init__(self, config=dict()):
        self.negasi = [line.replace('\n','') for line in open("negatingword.txt").read().splitlines()]
        self.tanya = [line.replace('\n','') for line in open("questionword.txt").read().splitlines()]
        #create sentiment words dictionary
        self.sentiwords_txt = [line.replace('\n','').split(":") for line in open("sentiwords_id.txt").read().splitlines()]
        self.sentiwords_dict = OrderedDict()
        for term in self.sentiwords_txt:
            self.sentiwords_dict[term[0]] = int(term[1])
        #create emoticon dictionary
        self.emoticon_txt = [line.replace('\n','').split(" | ") for line in open("emoticon_id.txt").read().splitlines()]
        self.emoticon_dict = OrderedDict()
        for term in self.emoticon_txt:
            self.emoticon_dict[term[0]] = int(term[1])
        #create idioms dictionary
        self.idioms_txt = [line.replace('\n','').split(":") for line in open("idioms_id.txt").read().splitlines()]
        self.idioms_dict = OrderedDict()
        for term in self.idioms_txt:
            self.idioms_dict[term[0]] = int(term[1])
        #create boosterwords dictionary
        self.boosterwords_txt = [line.replace('\n','').split(":") for line in open("boosterwords_id.txt").read().splitlines()]
        self.boosterwords_dict = OrderedDict()
        for term in self.boosterwords_txt:
            self.boosterwords_dict[term[0]] = int(term[1])
        self.negation_conf = config["negation"]
        self.booster_conf = config["booster"]
        self.ungkapan_conf = config["ungkapan"]
        self.consecutive_conf = config["consecutive"]
        self.repeated_conf = config["repeated"]
        self.emoticon_conf = config["emoticon"]
        self.question_conf = config["question"]
        self.exclamation_conf = config["exclamation"]
        self.punctuation_conf = config["punctuation"]
        self.mean_conf = False

    def senti(self,term):
        try:
            return self.sentiwords_dict[term]
        except:
            return 0

    def emosikon(self,term):
        try:
            return self.emoticon_dict[term]
        except:
            return 0

    def ungkapan(self,term):
        try:
            return self.idioms_dict[term]
        except:
            return 0

    def booster(self, term):
        try:
            return self.boosterwords_dict[term]
        except:
            return 0

    def cek_negationword(self, prev_term, prev_term2):
        #jika kata sebelumnya (index-1) adalah kata negasi, negasikan nilai -+nya
        if prev_term in self.negasi or prev_term2+" "+prev_term in self.negasi:
            # print prev_term
            self.score = -abs(self.score) if self.score>0 else abs(self.score)

    def cek_boosterword(self,term):
        booster_score = self.booster(term)
        if booster_score !=0 and self.score>0: self.score += booster_score
        if booster_score !=0 and self.score<0: self.score -= booster_score

    def cek_consecutive_term(self, prev_term):
        if self.prev_score>0 and self.score >=3: self.score+=1 
        if self.prev_score<0 and self.score <=-3: self.score-=1 

    def cek_ungkapan(self, bigram,trigram, i):
        bigram = ' '.join(bigram)
        trigram = ' '.join(trigram)
        ungkapan_score = self.ungkapan(bigram)
        if ungkapan_score==0:
            ungkapan_score = self.ungkapan(trigram)
        if ungkapan_score!=0:
            self.score = ungkapan_score
            self.prev_score = 0
            self.pre_max_pos[i-1] = 1
            self.pre_max_neg[i-1] = -1
            self.max_pos = self.pre_max_pos[i-2] #if len(self.pre_max_pos)>1 else 1
            self.max_neg = self.pre_max_neg[i-2] #if len(self.pre_max_neg)>1 else -1
            self.sentence_score[i-1] = re.sub(r'\[\d\]','',self.sentence_score[i-1])

    def cek_repeated_punctuation(self, next_term):
        if re.search(r'!{2,}',next_term) and self.score >=3: self.score+=1
        if re.search(r'!{2,}',next_term) and self.score <=-3: self.score-=1

    def remove_extra_repeated_char(self, term):
        return re.sub(r'([A-Za-z])\1{2,}',r'\1',term)
    def plural_to_singular(self, term):
        return re.sub(r'([A-Za-z]+)\-\1', r'\1',term)
    def classify(self):
        result = "netral"
        try:
            if self.mean_conf:
                mean_p = np.mean(self.mean_pos)
                mean_n = np.mean(self.mean_neg)
                print(mean_p, mean_n)
                if mean_p > mean_n:
                    result = "positif"
                elif mean_p < mean_n and not self.is_tanya:
                    result = "negatif"
                elif mean_p < mean_n and self.is_tanya:
                    result = "netral"
            else:
                if abs(self.sentences_max_pos) > abs(self.sentences_max_neg):
                    result = "positif"
                elif abs(self.sentences_max_pos) < abs(self.sentences_max_neg):
                    result = "negatif"
                elif abs(self.sentences_max_pos) == abs(self.sentences_max_neg):
                    result = "netral"
        except:
            print("error ",self.sentences_max_pos, self.sentences_max_neg)
        return result
    def cek_neutral_term(self,terms,i):
        if terms[i-1] in self.neutral_term or terms[i+1] in self.neutral_term: self.score=1 

    def main(self,sentence):
        self.neutral_term = ['jika','kalau']
        sentences = sentence.split('.')
        self.sentences_max_neg = -1
        self.sentences_max_pos = 1
        self.sentences_score = []
        self.sentences_text = []
        for sentence in sentences:
            self.max_neg = -1
            self.max_pos = 1
            self.mean_neg = [1]
            self.mean_pos = [1]
            self.sentence_score=[]
            terms = sentence.split()
            # terms = re.split(r'[\s,.]',sentence)
            terms_length = len(terms)
            self.is_tanya = False
            self.sentence_text = ''
            # print self.max_pos, self.max_neg
            #SEMUA KALIMAT YANG MEMILIKI TANDA SERU MEMILIKI +ve minimal 2
            if self.exclamation_conf and re.search('!',sentence): self.max_pos = 2
            self.prev_score = 0
            self.pre_max_pos = []
            self.pre_max_neg = []
            for i,term in enumerate(terms):
                # repeated_term = ''
                is_extra_char = False
                plural = ''
                self.score = 0
                # if re.search(r'[A-Za-z\-.]+',term):
                # print term
                if re.search(r'([A-Za-z])\1{3,}',term):
                    is_extra_char = True
                    # repeated_term =term
                term = self.remove_extra_repeated_char(term)
                if re.search(r'([A-Za-z]+)\-\1',term):
                    plural = term
                    term = self.plural_to_singular(term)
                #GET SENTI SCORE#
                self.score = self.senti(term)
                # print "senti score",term, self.score

                #NEGATION HANDLER#
                if self.negation_conf and self.score !=0 and i>0:self.cek_negationword(terms[i-1],terms[i-2])
                # print  "negation score",term, self.score

                #BOOSTERWORD HANDLER#
                if self.booster_conf and self.score !=0 and i>0 and i<=(terms_length-1):self.cek_boosterword(terms[i-1])
                if self.booster_conf and self.score !=0 and i>=0 and i<(terms_length-1):self.cek_boosterword(terms[i+1])
                # print  "booster score",term, self.score

                #IDIOM/UNGKAPAN HANDLER#
                if self.ungkapan_conf and i>0 and i<=(terms_length-1):self.cek_ungkapan([terms[i-1],term],[terms[i-2],terms[i-1],term],i)
                # if self.ungkapan_conf and i>=0 and i<(terms_length-1):self.cek_ungkapan([term,terms[i+1]])
                # print  "idiom score",term, self.score

                #CONSECUTIVE SENTIMENT WORD#
                if self.consecutive_conf and i>0 and i<=(terms_length-1) and self.score !=0:self.cek_consecutive_term(terms[i-1])
                # print  "consecutive score",term, self.score

                #+1 SENTI SCORE IF REPEATED CHAR ON POSITIVE/NEGATIVE +2 IF NEUTRAL TERM
                if self.repeated_conf and is_extra_char==True and self.score>0: self.score+=1
                if self.repeated_conf and is_extra_char==True and self.score<0: self.score-=1
                if self.repeated_conf and is_extra_char==True and self.score==0: self.score=2
                # print  "repeat char score", term, self.score
                if self.punctuation_conf and i>=0 and i<(terms_length-1): self.cek_repeated_punctuation(terms[i+1])
                # CEK APAKAH TERDAPAT KATA TANYA
                if self.question_conf and (term in self.tanya or re.search(r'\?',term)):self.is_tanya = True
                # CEK neutral term 
                if self.score!=0 and i>1 and i<(terms_length-2): self.cek_neutral_term(terms,i)
                # if self.score!=0 and i>0 and i<(terms_length-4): self.cek_neutral_term(terms,i)
                if self.emoticon_conf and self.score==0: self.score = self.emosikon(term)

                self.prev_score = self.score
                if self.mean_conf and self.score>0: self.mean_pos.append(self.score)	
                if self.mean_conf and self.score<0: self.mean_neg.append(abs(self.score))
                #GET MAX SCORE +ve/-ve	
                self.max_pos= self.score if self.score > self.max_pos else self.max_pos
                self.max_neg= self.score if self.score < self.max_neg else self.max_neg
                #insert score info current term
                self.pre_max_pos.append(self.max_pos)
                self.pre_max_neg.append(self.max_neg)
                # print self.pre_max_pos, self.pre_max_neg
                if plural !='': term = plural
                self.sentence_text += ' {}'.format(term)
                if self.score != 0:term = "{} [{}]".format(term, self.score)
                self.sentence_score.append(term)

            self.sentences_text.append(self.sentence_text)
            self.sentences_score.append(" ".join(self.sentence_score))
            if self.is_tanya: 
                self.max_neg = -1
            self.sentences_max_pos = self.max_pos if self.max_pos > self.sentences_max_pos else self.sentences_max_pos
            self.sentences_max_neg = self.max_neg if self.max_neg < self.sentences_max_neg else self.sentences_max_neg
            # print self.sentences_max_pos, self.sentences_max_neg
        sentence_result = self.classify()
        # print self.sentences_text
        return {"classified_text":". ".join(self.sentences_score),"tweet_text":". ".join(self.sentences_text),"sentence_score":self.sentences_score,"max_positive":self.sentences_max_pos,"max_negative":self.sentences_max_neg,"kelas":sentence_result}

In [2]:
config = dict()
config["negation"] = True
config["booster"]  = True
config["ungkapan"]  = True
config["consecutive"]  = True
config["repeated"]  = True
config["emoticon"]  = True
config["question"]  = True
config["exclamation"]  = True
config["punctuation"]  = True

In [3]:
senti = sentistrength(config)

In [4]:
# Load data ke dalam data frame 
import pandas as pd

# spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('lemma_200.csv', encoding='latin-1')

df.head()

,Unnamed: 0,Review,Sentimen,Review_norm,review_tokens,review_tokens_SR,review_tokens_stemmed
0,0,kecewa banget saya pesan yang free pouch tapi ...,Negatif,kecewa banget saya pesan yang free pouch tapi ...,"['kecewa', 'banget', 'saya', 'pesan', 'yang', ...","['kecewa', 'banget', 'pesan', 'free', 'pouch',...","['kecewa', 'banget', 'pesan', 'free', 'pouch',..."
1,1,kali ini pengemasan tidak safe sepertinya tutu...,Negatif,kali ini pengemasan tidak safe sepertinya tutu...,"['kali', 'ini', 'pengemasan', 'tidak', 'safe',...","['kali', 'pengemasan', 'tidak', 'safe', 'seper...","['kali', 'emas', 'tidak', 'safe', 'seperti', '..."
2,2,paket datangnya barang sudah pecah dan sy retu...,Negatif,paket datangnya barang sudah pecah dan saya re...,"['paket', 'datangnya', 'barang', 'sudah', 'pec...","['paket', 'datangnya', 'barang', 'pecah', 'ret...","['paket', 'datang', 'barang', 'pecah', 'return..."
3,3,ga ada sabun nya ga sesuai deskripsi,Negatif,enggak ada sabun nya enggak sesuai deskripsi,"['enggak', 'ada', 'sabun', 'nya', 'enggak', 's...","['enggak', 'sabun', 'nya', 'enggak', 'sesuai',...","['enggak', 'sabun', 'nya', 'enggak', 'sesuai',..."
4,4,ga sesuai harapan,Negatif,enggak sesuai harapan,"['enggak', 'sesuai', 'harapan']","['enggak', 'sesuai', 'harapan']","['enggak', 'sesuai', 'harap']"


In [5]:
# Drop kolom yang tidak terpakai
df = df.drop(df.columns[[0, 1, 3, 4, 5]], axis=1)

# Cek data teratas
df.head()

,Sentimen,review_tokens_stemmed
0,Negatif,"['kecewa', 'banget', 'pesan', 'free', 'pouch',..."
1,Negatif,"['kali', 'emas', 'tidak', 'safe', 'seperti', '..."
2,Negatif,"['paket', 'datang', 'barang', 'pecah', 'return..."
3,Negatif,"['enggak', 'sabun', 'nya', 'enggak', 'sesuai',..."
4,Negatif,"['enggak', 'sesuai', 'harap']"


In [6]:
# Mengubah data kolom menjadi kalimat utuh tanpa tanda koma, petik, dan tanda kurung siku
df['review_tokens_stemmed'] = df['review_tokens_stemmed'].apply(lambda x: ' '.join(eval(x)))

# Menampilkan hasil
df.head()

,Sentimen,review_tokens_stemmed
0,Negatif,kecewa banget pesan free pouch kirim cotton pa...
1,Negatif,kali emas tidak safe seperti tutup botol tidak...
2,Negatif,paket datang barang pecah return seller ikut k...
3,Negatif,enggak sabun nya enggak sesuai deskripsi
4,Negatif,enggak sesuai harap


In [7]:
# Memindahkan kolom sentimen setelah kolom review
columns = list(df.columns)
sentimen_index = columns.index('Sentimen')
review_index = columns.index('review_tokens_stemmed')
columns.insert(review_index + 1, columns.pop(sentimen_index))

# Menggunakan reindex untuk mengubah urutan kolom
df = df.reindex(columns=columns)

# Cek data teratas
df.head()


,review_tokens_stemmed,Sentimen
0,kecewa banget pesan free pouch kirim cotton pa...,Negatif
1,kali emas tidak safe seperti tutup botol tidak...,Negatif
2,paket datang barang pecah return seller ikut k...,Negatif
3,enggak sabun nya enggak sesuai deskripsi,Negatif
4,enggak sesuai harap,Negatif


In [8]:
# import hasil menjadi csv
df.to_csv("data200_new.csv", index=False)

In [9]:
import pandas as pd

# Membaca file CSV
df = pd.read_csv('data200_new.csv')

# Nama kolom yang ingin diubah dan dianalisis sentimennya
nama_kolom = 'review_tokens_stemmed'

# Fungsi untuk mendapatkan sentimen dari setiap baris dalam kolom
def analisis_sentimen(sentence):
    result = senti.main(sentence)
    return result

# Menggunakan fungsi apply untuk menerapkan analisis sentimen pada setiap baris
df['sentimen'] = df[nama_kolom].apply(analisis_sentimen)

# Menyimpan perubahan ke dalam file CSV
df.to_csv('sentimen_200.csv', index=False)

# Menampilkan DataFrame setelah perubahan
print(df)


                                 review_tokens_stemmed Sentimen  \
0    kecewa banget pesan free pouch kirim cotton pa...  Negatif   
1    kali emas tidak safe seperti tutup botol tidak...  Negatif   
2    paket datang barang pecah return seller ikut k...  Negatif   
3             enggak sabun nya enggak sesuai deskripsi  Negatif   
4                                  enggak sesuai harap  Negatif   
..                                                 ...      ...   
181  kirim emas cepat banget aman produk sesuai per...   Netral   
182                                                top  Positif   
183  tipe kulit minyak tipe kulit kering tipe kulit...  Positif   
184                                                exp  Positif   
185  pertama banget dulu sih beli bagus banget muka...   Netral   

                                              sentimen  
0    {'classified_text': 'kecewa [-4] banget pesan ...  
1    {'classified_text': 'kali emas [5] tidak safe ...  
2    {'classified_text':

In [10]:
import pandas as pd
import json

# Membaca file CSV
df = pd.read_csv('sentimen_200.csv')

# Nama kolom yang berisi data sentimen
nama_kolom_sentimen = 'sentimen'

# Fungsi untuk memisahkan data JSON dalam kolom sentimen
def parse_sentimen(sentimen_str):
    try:
        sentimen_dict = json.loads(sentimen_str.replace("'", "\""))
        return pd.Series(sentimen_dict)
    except:
        return pd.Series({})

# Memisahkan data dalam kolom 'sentimen' menjadi kolom-kolom terpisah
df_sentimen = df[nama_kolom_sentimen].apply(parse_sentimen)

# Menggabungkan DataFrame baru dengan DataFrame asli
df = pd.concat([df, df_sentimen], axis=1)

# Menyimpan perubahan ke dalam file CSV
df.to_csv('sentimen_200_terpisah.csv', index=False)

# Menampilkan DataFrame setelah perubahan
print(df)


                                 review_tokens_stemmed Sentimen  \
0    kecewa banget pesan free pouch kirim cotton pa...  Negatif   
1    kali emas tidak safe seperti tutup botol tidak...  Negatif   
2    paket datang barang pecah return seller ikut k...  Negatif   
3             enggak sabun nya enggak sesuai deskripsi  Negatif   
4                                  enggak sesuai harap  Negatif   
..                                                 ...      ...   
181  kirim emas cepat banget aman produk sesuai per...   Netral   
182                                                top  Positif   
183  tipe kulit minyak tipe kulit kering tipe kulit...  Positif   
184                                                exp  Positif   
185  pertama banget dulu sih beli bagus banget muka...   Netral   

                                              sentimen  \
0    {'classified_text': 'kecewa [-4] banget pesan ...   
1    {'classified_text': 'kali emas [5] tidak safe ...   
2    {'classified_tex

### CEK JUMLAH REVIEW SETIAP KELAS

In [11]:
# Load data ke dalam data frame 
import pandas as pd

# spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('sentimen_200_terpisah.csv', encoding='latin-1') 

df.head()

,review_tokens_stemmed,Sentimen,sentimen,classified_text,tweet_text,sentence_score,max_positive,max_negative,kelas
0,kecewa banget pesan free pouch kirim cotton pa...,Negatif,{'classified_text': 'kecewa [-4] banget pesan ...,kecewa [-4] banget pesan free pouch kirim cott...,kecewa banget pesan free pouch kirim cotton p...,['kecewa [-4] banget pesan free pouch kirim co...,5,-4,positif
1,kali emas tidak safe seperti tutup botol tidak...,Negatif,{'classified_text': 'kali emas [5] tidak safe ...,kali emas [5] tidak safe seperti [1] tutup bot...,kali emas tidak safe seperti tutup botol tida...,['kali emas [5] tidak safe seperti [1] tutup b...,5,-3,positif
2,paket datang barang pecah return seller ikut k...,Negatif,{'classified_text': 'paket datang barang pecah...,paket datang barang pecah [-2] return seller i...,paket datang barang pecah return seller ikut ...,['paket datang barang pecah [-2] return seller...,1,-1,netral
3,enggak sabun nya enggak sesuai deskripsi,Negatif,{'classified_text': 'enggak sabun nya enggak s...,enggak sabun nya enggak sesuai [3] deskripsi,enggak sabun nya enggak sesuai deskripsi,['enggak sabun nya enggak sesuai [3] deskripsi'],3,-1,positif
4,enggak sesuai harap,Negatif,"{'classified_text': 'enggak sesuai [3] harap',...",enggak sesuai [3] harap,enggak sesuai harap,['enggak sesuai [3] harap'],3,-1,positif


In [12]:
# Drop kolom yang tidak terpakai
df = df.drop(df.columns[[2, 3, 4, 5, 6, 7]], axis=1)

# Cek data teratas
df.head()

,review_tokens_stemmed,Sentimen,kelas
0,kecewa banget pesan free pouch kirim cotton pa...,Negatif,positif
1,kali emas tidak safe seperti tutup botol tidak...,Negatif,positif
2,paket datang barang pecah return seller ikut k...,Negatif,netral
3,enggak sabun nya enggak sesuai deskripsi,Negatif,positif
4,enggak sesuai harap,Negatif,positif


In [13]:
# Data untuk rename kolom v1 dan v2
new_cols = {
    'review_tokens_stemmed' : 'Review',
    'Sentimen' : 'Sentimen-Manual',
    'kelas' : 'Sentimen-SentiStrength'
}

# Rename nama kolom v1 dan v2
df = df.rename(columns=new_cols)

# Cek data teratas
df.head()

,Review,Sentimen-Manual,Sentimen-SentiStrength
0,kecewa banget pesan free pouch kirim cotton pa...,Negatif,positif
1,kali emas tidak safe seperti tutup botol tidak...,Negatif,positif
2,paket datang barang pecah return seller ikut k...,Negatif,netral
3,enggak sabun nya enggak sesuai deskripsi,Negatif,positif
4,enggak sesuai harap,Negatif,positif


In [14]:
# Cek jumlah data per kelas
print(df['Sentimen-SentiStrength'].value_counts())
print('\n')

# Cek kelengkapan data
print(df.info())
print('\n')

# Cek statistik deskriptif
print(df.describe())

positif    107
netral      47
negatif     32
Name: Sentimen-SentiStrength, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Review                  186 non-null    object
 1   Sentimen-Manual         186 non-null    object
 2   Sentimen-SentiStrength  186 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB
None


              Review Sentimen-Manual Sentimen-SentiStrength
count            186             186                    186
unique           182               3                      3
top     terima kasih         Positif                positif
freq               3              71                    107
